In [8]:
import pandas as pd
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.pipeline import Pipeline
import numpy as np
# Load the datasets
transactions_path = '/content/Transactions.csv'
products_path = '/content/Products.csv'
customers_path = '/content/Customers.csv'

transactions = pd.read_csv(transactions_path)
products = pd.read_csv(products_path)
customers = pd.read_csv(customers_path)


In [2]:
# Inspect the data
transactions.head(4),products.head(4), customers.head(4)

(  TransactionID CustomerID ProductID      TransactionDate  Quantity  \
 0        T00001      C0199      P067  2024-08-25 12:38:23         1   
 1        T00112      C0146      P067  2024-05-27 22:23:54         1   
 2        T00166      C0127      P067  2024-04-25 07:38:55         1   
 3        T00272      C0087      P067  2024-03-26 22:55:37         2   
 
    TotalValue   Price  
 0      300.68  300.68  
 1      300.68  300.68  
 2      300.68  300.68  
 3      601.36  300.68  ,
   ProductID              ProductName     Category   Price
 0      P001     ActiveWear Biography        Books  169.30
 1      P002    ActiveWear Smartwatch  Electronics  346.30
 2      P003  ComfortLiving Biography        Books   44.12
 3      P004            BookWorld Rug   Home Decor   95.69,
   CustomerID        CustomerName         Region  SignupDate
 0      C0001    Lawrence Carroll  South America  2022-07-10
 1      C0002      Elizabeth Lutz           Asia  2022-02-13
 2      C0003      Michael Rivera

In [3]:
# Merge transactions with products on ProductID
transactions_products = pd.merge(transactions, products, on="ProductID", how="left")

# Merge the result with customers on CustomerID
data = pd.merge(transactions_products, customers, on="CustomerID", how="left")

# Preview the merged dataset
data.head()

,TransactionID,CustomerID,ProductID,TransactionDate,Quantity,TotalValue,Price_x,ProductName,Category,Price_y,CustomerName,Region,SignupDate
0,T00001,C0199,P067,2024-08-25 12:38:23,1,300.68,300.68,ComfortLiving Bluetooth Speaker,Electronics,300.68,Andrea Jenkins,Europe,2022-12-03
1,T00112,C0146,P067,2024-05-27 22:23:54,1,300.68,300.68,ComfortLiving Bluetooth Speaker,Electronics,300.68,Brittany Harvey,Asia,2024-09-04
2,T00166,C0127,P067,2024-04-25 07:38:55,1,300.68,300.68,ComfortLiving Bluetooth Speaker,Electronics,300.68,Kathryn Stevens,Europe,2024-04-04
3,T00272,C0087,P067,2024-03-26 22:55:37,2,601.36,300.68,ComfortLiving Bluetooth Speaker,Electronics,300.68,Travis Campbell,South America,2024-04-11
4,T00363,C0070,P067,2024-03-21 15:10:10,3,902.04,300.68,ComfortLiving Bluetooth Speaker,Electronics,300.68,Timothy Perez,Europe,2022-03-15


In [4]:
# Convert TransactionDate to datetime
data["TransactionDate"] = pd.to_datetime(data["TransactionDate"])

In [5]:
# Aggregate data
customer_features = data.groupby("CustomerID").agg(
    total_spend=("TotalValue", "sum"),
    purchase_count=("TransactionID", "count"),
    avg_transaction_value=("TotalValue", "mean"),
    first_purchase=("TransactionDate", "min"),
    last_purchase=("TransactionDate", "max"),
    preferred_category=("Category", lambda x: x.mode()[0] if not x.mode().empty else None),
    region=("Region", "first")
).reset_index()

In [6]:
# Calculate purchase frequency (days between first and last purchase)
customer_features["purchase_frequency"] = (
    (customer_features["last_purchase"] - customer_features["first_purchase"]).dt.days
    / customer_features["purchase_count"]
).fillna(0)

In [7]:
# Drop redundant date columns for simplicity
customer_features.drop(columns=["first_purchase", "last_purchase"], inplace=True)

# Preview the engineered features
customer_features.head()

,CustomerID,total_spend,purchase_count,avg_transaction_value,preferred_category,region,purchase_frequency
0,C0001,3354.52,5,670.904,Electronics,South America,57.600000
1,C0002,1862.74,4,465.685,Clothing,Asia,69.500000
2,C0003,2725.38,4,681.345,Home Decor,South America,47.000000
3,C0004,5354.88,8,669.360,Books,South America,37.375000
4,C0005,2034.24,3,678.080,Electronics,Asia,77.666667


In [9]:
# Normalize and Encode Features
numeric_features = ["total_spend", "purchase_count", "avg_transaction_value", "purchase_frequency"]
categorical_features = ["preferred_category", "region"]

# Define transformations
numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder()

# Create preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features)
    ]
)


In [10]:
# Apply transformations
processed_features = preprocessor.fit_transform(customer_features)

# Convert processed data back to DataFrame
processed_feature_names = (
    numeric_features
    + list(preprocessor.named_transformers_["cat"].get_feature_names_out(categorical_features))
)
customer_features_normalized = pd.DataFrame(
    processed_features,
    columns=processed_feature_names,
    index=customer_features["CustomerID"]
)


In [13]:
#Create Lookalike Map
def get_top_lookalikes(customers_df, target_ids, top_n=3):
    # Compute cosine similarity
    similarity_matrix = cosine_similarity(customers_df)
    similarity_df = pd.DataFrame(similarity_matrix, index=customers_df.index, columns=customers_df.index)

    lookalike_map = {}
    for target_customer_id in target_ids:
        if target_customer_id not in customers_df.index:
            continue

        # Get similarity scores for the target customer
        target_scores = similarity_df.loc[target_customer_id]

        # Sort and return top N similar customers (excluding the target itself)
        similar_customers = target_scores.drop(target_customer_id).nlargest(top_n)
        lookalike_map[target_customer_id] = list(similar_customers.items())

    return lookalike_map

In [15]:
# Generate the lookalike map for customers C0001 to C0020
target_customer_ids = [f"C{i:04d}" for i in range(1, 21)]
lookalike_map = get_top_lookalikes(customer_features_normalized, target_customer_ids)



In [19]:
# Save the lookalike map to Lookalike.csv
import csv
output_file = "/content/Lookalike.csv"
with open(output_file, mode="w", newline="") as file:
    writer = csv.writer(file) # Now csv.writer is accessible
    writer.writerow(["cust_id", "lookalikes"])
    for cust_id, lookalikes in lookalike_map.items():
        lookalike_str = "; ".join([f"{c}, {s:.4f}" for c, s in lookalikes])
        writer.writerow([cust_id, lookalike_str])

print(f"Lookalike map saved to {output_file}")

Lookalike map saved to /content/Lookalike.csv
